First, read all the lines in train.csv

In [1]:
import csv
raw_input_file = open('train.csv')
raw_input = csv.reader(raw_input_file)
lines = []
for row in raw_input:
    lines += [row]
raw_input_file.close()

Transform csv into proper format for xgboost, and divide labeled data into training and validation set.

use datetime, chinese_holiday and workalendar to deal with the datetime field.

The features I use:
    Year, month, day, hour
    Day of week(Monday - Sunday)
    Is the day a holiday of Hong Kong or Mainland China or not

One-hot everything!

Use random shuffle to divide the initial dataset into training and validation.

In [2]:
import datetime
import chinese_holiday
from workalendar.asia import HongKong
calendar = HongKong()
input_for_xgb = []
for line in lines[1:]:
    temp = [float(line[2])]
    date_time_obj = datetime.datetime.strptime(line[1], '%d/%m/%Y %H:%M')
    temp_year = date_time_obj.year
    temp_month = date_time_obj.month
    temp_day = date_time_obj.day
    temp_hour = date_time_obj.hour
    temp_weekday = date_time_obj.weekday()
    temp += [int(calendar.is_holiday(date_time_obj))]
    temp += [int(chinese_holiday.is_holiday(date_time_obj))]
    temp += [0 if temp_year == 2017 else 1]
    temp += [1 if i == temp_month else 0 for i in range(1,13)]
    temp += [1 if i == temp_day else 0 for i in range(1,32)]
    temp += [1 if i == temp_hour else 0 for i in range(24)]
    temp += [1 if i == temp_weekday else 0 for i in range(7)]
    input_for_xgb += [temp]
import random
random.shuffle(input_for_xgb)
xgb_input_file = open('xgb_train.txt','w')
for line in input_for_xgb[:10000]:
    xgb_input_file.write(str(line[0]))
    xgb_input_file.write(' ')
    for i in range(1,len(input_for_xgb[0])):
        xgb_input_file.write(str(i))
        xgb_input_file.write(':')
        xgb_input_file.write(str(line[i]))
        xgb_input_file.write(' ')
    xgb_input_file.write('\n')
xgb_input_file.close()
xgb_input_file = open('xgb_val.txt','w')
for line in input_for_xgb[10000:]:
    xgb_input_file.write(str(line[0]))
    xgb_input_file.write(' ')
    for i in range(1,len(input_for_xgb[0])):
        xgb_input_file.write(str(i))
        xgb_input_file.write(':')
        xgb_input_file.write(str(line[i]))
        xgb_input_file.write(' ')
    xgb_input_file.write('\n')
xgb_input_file.close()

Read input files into xgboost and train a random forest model.
Although the official guide says that learning_rate and num_boost_round must be 1, I found that tuning them will yield better results.

In [4]:
import xgboost as xgb
dtrain = xgb.DMatrix('xgb_train.txt')
dval = xgb.DMatrix('xgb_val.txt')
params = {
  'colsample_bynode': 0.4,
  'learning_rate': 0.8,
  'max_depth': 6,
  'num_parallel_tree': 80,
  'objective': 'reg:squarederror',
  'subsample': 0.6,
  'tree_method': 'gpu_hist',
}
bst = xgb.train(params, dtrain, num_boost_round=400)
preds = bst.predict(dtrain)
val_preds = bst.predict(dval)

Xgboost does not have output while training, so I have to manually output the mean squared error of training and validation set.

In [5]:
real = [line[0] for line in input_for_xgb]
real_train = real[:10000]
real_val = real[10000:]
total_se = 0
for i in range(len(real_train)):
    total_se += (real_train[i]-preds[i])**2
mse = total_se/len(real_train)
print('train:',mse)
total_se = 0
for i in range(len(real_val)):
    total_se += (real_val[i]-val_preds[i])**2
mse = total_se/len(real_val)
print('val:',mse)

train: 9.663658442125065
val: 14.57769868472649


All the cells below just use the model I trained above to produce the submission file.

In [6]:
raw_output_file = open('test.csv')
raw_output = csv.reader(raw_output_file)
lines = []
for row in raw_output:
    lines += [row]
raw_output_file.close()

In [7]:
input_for_xgb = []
for line in lines[1:]:
    temp = [0]
    date_time_obj = datetime.datetime.strptime(line[1], '%d/%m/%Y %H:%M')
    temp_year = date_time_obj.year
    temp_month = date_time_obj.month
    temp_day = date_time_obj.day
    temp_hour = date_time_obj.hour
    temp_weekday = date_time_obj.weekday()
    temp += [int(calendar.is_holiday(date_time_obj))]
    temp += [int(chinese_holiday.is_holiday(date_time_obj))]
    temp += [0 if temp_year == 2017 else 1]
    temp += [1 if i == temp_month else 0 for i in range(1,13)]
    temp += [1 if i == temp_day else 0 for i in range(1,32)]
    temp += [1 if i == temp_hour else 0 for i in range(24)]
    temp += [1 if i == temp_weekday else 0 for i in range(7)]
    input_for_xgb += [temp]
xgb_input_file = open('xgb_test.txt','w')
for line in input_for_xgb:
    xgb_input_file.write(str(line[0]))
    xgb_input_file.write(' ')
    for i in range(1,len(input_for_xgb[0])):
        xgb_input_file.write(str(i))
        xgb_input_file.write(':')
        xgb_input_file.write(str(line[i]))
        xgb_input_file.write(' ')
    xgb_input_file.write('\n')
xgb_input_file.close()

In [8]:
dtest = xgb.DMatrix('xgb_test.txt')
test_preds = bst.predict(dtest)
test_preds

array([50.098682, 48.728905, 35.66695 , ..., 44.882774, 39.332832,
       42.395332], dtype=float32)

In [9]:
submit = open('submit.csv','w')
submit.write('id,speed\n')
for i in range(len(test_preds)):
    submit.write(str(i))
    submit.write(',')
    submit.write(str(test_preds[i]))
    if(i!=len(test_preds)-1):
        submit.write('\n')
submit.close()